# Reconstructing a <i>Mimulus naiandinus</i> genome from hybrid genomes

## <a id="contents">Table o' contents</a>

[Look at read qualities](#read_quality)

[Align our reads to the reference genome](#align)

[Filtering and Processing alignments](#FandP)
  - [Convert SAMs to BAMs](#sam2bam)
  - [Reorder alignments to reference](#reorder)
  - [DeDuplicate reads](#dedupe)
  - [Merging BAM files](#mergeBAM)

## <a id="read_quality">Look at read qualities</a>

Let's use fastx tools to take a look at the average read qualities.

In [6]:
## path to our fastq files
fpath='/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/'

Fastx requires that we calculate the statistics first:

In [5]:
for i in $fpath*"fastq"
do
echo $i
$ftd"fastx_quality_stats" -Q33 -i $i -o $i"-Stats.txt"
echo $i done!
done


/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S1_S16_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S1_S16_L002_R1_001.fastq done!
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S2_S17_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S2_S17_L002_R1_001.fastq done!
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S3_S18_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S3_S18_L002_R1_001.fastq done!
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S4_S19_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S4_S19_L002_R1_001.fastq done!
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S5_S20_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S5_S20_L002

Then graph:

In [8]:
for i in $fpath*Stats.txt; do
fastq_quality_boxplot_graph.sh -i $i -o $i"quality.eps" -p
echo $i"quality.eps"
done


/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S1_S16_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S2_S17_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S3_S18_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S4_S19_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S5_S20_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S6_S21_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/merged.fastq-Stats.txtquality.eps


What do these look like? Here is our merged file, that has all of the reads:

![read_quality](merged.fastq-Stats.txtquality.png)

Looks great, high quality reads, with almost no variance. 

## <a id="align">Align our reads to the reference genome</a>

We'll use [bowtie2](http://bowtie-bio.sourceforge.net/bowtie2/index.shtml) to align our reads to our reference genome. The reference genome needs to be indexed in the way that bowtie2 wants, first:

In [10]:
bowtie2-build -f /Users/danthomas/Documents/naiandinus_gen/Luteus_genome/Mimulus_luteus.fasta Mlu

Settings:
  Output files: "Mlu.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /Users/danthomas/Documents/naiandinus_gen/Luteus_genome/Mimulus_luteus.fasta
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:09
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:04
bmax according to bmaxDivN setting: 98519930
Using parameters --bmax 73889948 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these paramet

Then we can loop through our fastq files and do alignments against this reference genome. For the moment, we'll do these as individual alignments of the six bins of phenotypes assigned by Arielle and Josh. This results in a sequence alignment map (SAM) for each of the six.

In [11]:
cd /Users/danthomas/Documents/naiandinus_gen/Luteus_genome

for i in $fpath*001.fastq; do
    j=$(basename $i)
    k=${j/fastq/sam}
    bowtie2 --local  -x Mlu -U $i -S /Users/danthomas/Documents/naiandinus_gen/SAM/$k \
done

Some output:

In [23]:
cat bowtie_log.txt

56034871 reads; of these:
  56034871 (100.00%) were unpaired; of these:
    7893494 (14.09%) aligned 0 times
    11729089 (20.93%) aligned exactly 1 time
    36412288 (64.98%) aligned >1 times
85.91% overall alignment rate
68049510 reads; of these:
  68049510 (100.00%) were unpaired; of these:
    7592098 (11.16%) aligned 0 times
    14879687 (21.87%) aligned exactly 1 time
    45577725 (66.98%) aligned >1 times
88.84% overall alignment rate
60681218 reads; of these:
  60681218 (100.00%) were unpaired; of these:
    9222716 (15.20%) aligned 0 times
    12679236 (20.89%) aligned exactly 1 time
    38779266 (63.91%) aligned >1 times
84.80% overall alignment rate
55885597 reads; of these:
  55885597 (100.00%) were unpaired; of these:
    7485075 (13.39%) aligned 0 times
    11651355 (20.85%) aligned exactly 1 time
    36749167 (65.76%) aligned >1 times
86.61% overall alignment rate
67627796 reads; of these:
  67627796 (100.00%) were unpaired; of these:
    15149703 (22.40%) aligned 0 time

Time for these alignments, as a whole:

real    565m3.770s  
user    554m22.225s  
sys     4m27.857s


These SAM files look like this, a header that lists out the lengths of our scaffolds, and a body that is a long list of where the various reads were assigned on the reference genome, with a mapq quality score for the confidence in the placement of this read on the reference genome, the quality of the basecalls, other good stuff. More info <a href="https://en.wikipedia.org/wiki/SAM_(file_format)">here</a>.

In [17]:
head JP-S6_S21_L002_R1_001.sam

@HD	VN:1.0	SO:unsorted
@SQ	SN:lcl|scaffold_32	LN:817265
@SQ	SN:lcl|scaffold_21	LN:849581
@SQ	SN:lcl|scaffold_14	LN:884134
@SQ	SN:lcl|scaffold_37	LN:783220
@SQ	SN:lcl|scaffold_23	LN:851886
@SQ	SN:lcl|scaffold_8	LN:980590
@SQ	SN:lcl|scaffold_9	LN:976062
@SQ	SN:lcl|scaffold_33	LN:797088
@SQ	SN:lcl|scaffold_38	LN:791999


In [18]:
tail JP-S6_S21_L002_R1_001.sam

K00282:27:HC3GVBBXX:2:2228:30959:49194	0	lcl|scaffold_606	126980	44	48M3S	*	0	0	TTGAGCCTTTTGTAGTAGGATTTATGAATTTCTCTGGAGCTTTATTAGACT	AAFFFJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ	AS:i:96	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:48	YT:Z:UU
K00282:27:HC3GVBBXX:2:2228:31081:49194	16	lcl|scaffold_24	610319	11	1S33M1I14M2S	*	0	0	CCATGTTGGAGTCCGAGTTTTGGAATGTTGCATGAAATAAGAATTGTAGAC	JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFFFAA	AS:i:78	XS:i:76	XN:i:0	XM:i:1	XO:i:1	XG:i:1	NM:i:2	MD:Z:30G16	YT:Z:UU
K00282:27:HC3GVBBXX:2:2228:31284:49194	16	lcl|scaffold_1046	13016	17	51M	*	0	0	CCGCGATTTCTCTCGATTTCTAATGCAATACACAAATCAATTGGTTCTGTT	JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFFFAA	AS:i:94	XS:i:80	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:24T26	YT:Z:UU
K00282:27:HC3GVBBXX:2:2228:31527:49194	0	lcl|scaffold_448	249337	31	51M	*	0	0	AGTAGCATGGGCTGCAATAAAGGCCCGGTGTCATTGTGTTAATAAAAAATG	AAFFFJJJJJJJJJJJJ<JJJJJJJJJJJJJJFJJJJJJJJJJJJJJJJJJ	AS:i:102	XS:i:94	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:51	Y

## <a id=FandP>Filtering and Processing alignments</a>

Before calling our variants (places where our new genomes are different from the reference), we've got some cleanup to do to our new alignments.

### <a id="sam2bam">Convert SAMs to BAMs</a>

For downstream work, we'll work with compressed SAM files, called BAM files. We'll use <a href="http://www.htslib.org">Samtools</a> for this and other tasks.

In [25]:
cd /Users/danthomas/Documents/naiandinus_gen/SAM

for i in *sam; do
    time samtools view $i -bS > ./BAM/${i/sam/bam} && \
    echo $i
done

JP-S1_S16_L002_R1_001.sam
JP-S2_S17_L002_R1_001.sam
JP-S3_S18_L002_R1_001.sam
JP-S4_S19_L002_R1_001.sam
JP-S5_S20_L002_R1_001.sam
JP-S6_S21_L002_R1_001.sam


## <a id="reorder">Reorder alignments to reference</a>

It's also necessary to sort the alignments in the order they appear on the reference genome. <a href="https://broadinstitute.github.io/picard/">Picard</a> tools are useful here. 

In [ ]:
cd /Users/danthomas/Documents/naiandinus_gen/BAM

for i in *
do 
    output="sortedBAM/"${i/.bam/.sorted.bam}
    picard SortSam I=$i O=$output SORT_ORDER=coordinate && \
    ls $i
done

## <a id="dedupe">DeDuplicate reads</a>

We can also use Picard tools to remove duplicate reads. This has been recommended to me, and it's discussed <a href="https://broadinstitute.github.io/picard/command-line-overview.html#MarkDuplicates">here</a>. 

I'm a little unclear on this. I can see why removing duplicates is handy.  Dereplicating the redundant information in duplicates makes sense as far as computational expense. But the explanation in picard's manual implies that all duplicates are erroroneous reads, either from bench preps or the cameras of the sequencers - how can this be true? Presumably the same digestion site occurs several times, gets sequenced? Maybe the sites of digestion are random, so it is really unlikely that a sequence would happen twice. If duplicate reads are real (can they be?) aren't we losing converage by removing them?

Not sure, anyway, just following orders like a good scientist...

In [ ]:
## home for our deduplicated maps:
ddm="/Users/danthomas/Documents/naiandinus_gen/BAM/dmBAM/"

cd /Users/danthomas/Documents/naiandinus_gen/BAM/sortedBAM

for i in *
do
    output=${i/.sorted.bam/.MD.bam}
    echo $output
    picard MarkDuplicates \
        I=$i \
        O=$ddm$output \
        M=$output.metrics.txt \
        REMOVE_DUPLICATES=true \
        MAX_FILE_HANDLES_FOR_READ_ENDS_MAP=256 && echo $i
done


We got the last input by asking our computer what kind of load it can handle at the moment, in terms of number of files:

In [28]:
ulimit -n 

256


### <a id="readGroups">Adding read group information to reads</a>

We're going to merge all of 6 of the genomes, that were binned by phenotype, for the variant calls. Before we do this, we want to retain the information about what phenotype a given read came from. In the fastq files and SAM/BAM files, this information is encoded in the read name. But from here this information is tracked as part of the "RG" info, or read group information, in the merged BAM file. Picard tools again:

In [ ]:
cd /Users/danthomas/Documents/naiandinus_gen/BAM/dmBAM

rgBAMs="/Users/danthomas/Documents/naiandinus_gen/BAM/rgBAM/"

touch $rgBAMs"rglog.txt"

for i in *; do
    echo $i >> $rgBAMs"rglog.txt"
    SM=$(echo $i | sed "s/.*_S/S/g" | sed "s/_L.*//g")
    output=$rgBAMs${i/.MD.bam/.RG.bam}
    picard AddOrReplaceReadGroups  I=$i  O=$output  RGPL=illumina  RGPU=indexSEQ  RGID=27  RGLB=A_Library  RGSM=$SM
    echo $output >> $rgBAMs"rglog.txt"
done


### <a id="mergeBAM">Merging BAM files</a>